# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import glob
import os

py.init_notebook_mode(connected=True)

# DataSet

In [2]:
db_data = pd.read_csv("OPENDATA_BOOKING_CARSHARING.csv", sep=';', header=0)
db_data[548069:]

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,START_RENTAL_ZONE,START_RENTAL_ZONE_HAL_ID,END_RENTAL_ZONE,END_RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL
548069,46385327,100001,149218,B45F2CFD95C5887698AD87B6596102E24DCCE243,2017-05-16 07:15:07,2017-05-16 07:15:00,2017-05-16 08:15:00,Nein,Ja,NaN,Sredzkistraße,402667,Sredzkistraße,402667,Station,Berlin,Flinkster iPhone
548070,46385351,100001,174116,954FB74EF255E4089D3D7DA3FE6234DE781BFDD3,2017-05-16 07:16:29,2017-05-16 08:30:00,2017-05-16 12:30:00,Nein,Nein,NaN,Bf Köln Messe/Deutz,400755,Bf Köln Messe/Deutz,400755,Station,Köln,Internet
548071,46385778,100001,173552,A986BBFA0F81DD3D2EB02A97A30A97CF111BD2C1,2017-05-16 07:37:19,2017-05-16 08:45:00,2017-05-18 09:45:00,Nein,Nein,NaN,Hbf Saarbrücken,400280,Hbf Saarbrücken,400280,Station,Saarbrücken,Internet
548072,46385814,100012,169106,1127C45D5E5C398FE0BCAF07D6C6694C2C9E0B0B,2017-05-16 07:39:22,2017-05-16 07:45:00,2017-05-16 10:45:00,Nein,Nein,NaN,Hbf Gelsenkirchen,406277,Hbf Gelsenkirchen,406277,Station,Gelsenkirchen,Internet


In [3]:
db_data['TRAVERSE'] = db_data['TRAVERSE_USE'].map({'Nein' : 0 , 'Ja':1}).astype(int)
db_data['EXTRA_BOOKING_FEE']= db_data['COMPUTE_EXTRA_BOOKING_FEE'].map({'Nein': 0 , 'Ja': 1}).astype(int)
#db_data['RENTAL_ZONE_HAL_SRC']= db_data['RENTAL_ZONE_HAL_SRC'].map({'Station': 1}).astype(int)
#db_data.head()

In [4]:
db_data['TECHNICAL_INCOME_CHANNEL'].fillna('UNKNOWN', inplace=True)

In [5]:
db_data = db_data[db_data.TECHNICAL_INCOME_CHANNEL != 'UNKNOWN']

In [6]:
db_data['DISTANCE'].fillna('UNKNOWN', inplace=True)

In [7]:
db_data = db_data[db_data.DISTANCE != 'UNKNOWN']

In [8]:
#db_data.isnull().sum()

In [9]:
#db_data.apply(lambda x: len(x.unique()))

In [10]:
cartype = pd.read_csv("OPENDATA_VEHICLE_CARSHARING.csv", sep=';', header=0)
columns = ['VEHICLE_MODEL_TYPE','VEHICLE_MODEL_NAME','VEHICLE_TYPE_NAME','VIN','REGISTRATION_PLATE','SERIAL_NUMBER','KW','OWNERSHIP_TYPE','CAPACITY_AMOUNT', 'ACCESS_CONTROL_COMPONENT_TYPE', 'COMPANY', 'COMPANY_GROUP'] 
cartype.drop(columns, inplace=True, axis=1)
#cartype.head()
carmerge = pd.merge(db_data, cartype, on ='VEHICLE_HAL_ID')
#carmerge.head()

In [11]:
#carmerge.apply(lambda x: len(x.unique()))

In [12]:
catgtype = pd.read_csv("OPENDATA_CATEGORY_CARSHARING.csv", sep=';', header=0)
columns = ['COMPANY','COMPANY_GROUP']
catgtype.drop(columns, inplace=True, axis=1)
catgtype = catgtype.rename(columns={'HAL_ID':'CATEGORY_HAL_ID'})
data_p = pd.merge(carmerge, catgtype, on ='CATEGORY_HAL_ID')
catgtype.head()

,CATEGORY_HAL_ID,CATEGORY
0,100000,Werbeklasse (mit Beklebung)
1,100001,Kleinklasse (teilweise ohne Navi)
2,100002,Mini (teilweise ohne Navi)
3,100003,Kompaktklasse
4,100004,Zubehör


In [13]:
#data_p.apply(lambda x: len(x.unique()))

In [14]:
#data_p.isnull().sum()

In [15]:
population = pd.read_csv("Germany_population_by_city_2015.csv", sep=',', header=0)
#population

In [16]:
population.apply(lambda x: len(x.unique()))

CITY_RENTAL_ZONE    85
POPULATION          85
FEDERAL_STATE       15
dtype: int64

In [17]:
data = pd.merge(data_p, population, on ='CITY_RENTAL_ZONE')
data.head(10)

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,...,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL,TRAVERSE,EXTRA_BOOKING_FEE,VEHICLE_MANUFACTURER_NAME,FUEL_TYPE_NAME,CATEGORY,POPULATION,FEDERAL_STATE
0,17842196,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-06-05 08:49:33,2014-01-12 13:00:00,2014-01-12 14:30:00,Nein,Nein,14,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
1,19923967,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-09-15 12:19:14,2014-01-05 14:00:00,2014-01-05 15:30:00,Nein,Nein,0,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
2,20453330,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-10-19 13:25:38,2014-03-16 18:15:00,2014-03-16 19:30:00,Nein,Nein,17,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
3,20799835,100012,150359,D564D3A1EA8BF096581556520B8F6582BEB65EFD,2013-11-10 14:10:26,2014-01-06 17:00:00,2014-01-06 23:45:00,Nein,Nein,16,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
4,20799864,100012,150359,D564D3A1EA8BF096581556520B8F6582BEB65EFD,2013-11-10 14:13:25,2014-01-13 17:00:00,2014-01-13 23:45:00,Nein,Nein,16,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
5,20799880,100012,150359,D564D3A1EA8BF096581556520B8F6582BEB65EFD,2013-11-10 14:14:54,2014-01-20 17:00:00,2014-01-20 23:45:00,Nein,Nein,15,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
6,20896382,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-11-17 12:42:57,2014-02-04 08:45:00,2014-02-04 10:15:00,Nein,Nein,5,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
7,20896766,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-11-17 13:20:34,2014-01-01 14:30:00,2014-01-01 16:00:00,Nein,Nein,16,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
8,21378191,100012,150359,0FC7F06DDDD951AF7009288F8D4B9060925ED107,2014-01-02 17:54:09,2014-01-03 09:00:00,2014-01-03 22:30:00,Nein,Nein,1001,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen
9,21394268,100012,150359,0646C8373257083080F514A1E7B483BE1020D949,2014-01-04 15:07:28,2014-01-04 16:30:00,2014-01-04 17:30:00,Nein,Nein,2,...,Station,Köln,Internet,0,0,Citroën,Diesel,Sonderklasse (Alfa MiTo / Citroën DS3),1060582,Nordrhein-Westfalen


# Converting to datetime type

In [18]:
data['DATE_BOOKING'] = pd.to_datetime(data.DATE_BOOKING)
data['DATE_FROM'] = pd.to_datetime(data.DATE_FROM)
data['DATE_UNTIL'] = pd.to_datetime(data.DATE_UNTIL)

In [19]:
data.dtypes

BOOKING_HAL_ID                        int64
CATEGORY_HAL_ID                       int64
VEHICLE_HAL_ID                        int64
CUSTOMER_HAL_ID                      object
DATE_BOOKING                 datetime64[ns]
DATE_FROM                    datetime64[ns]
DATE_UNTIL                   datetime64[ns]
COMPUTE_EXTRA_BOOKING_FEE            object
TRAVERSE_USE                         object
DISTANCE                             object
START_RENTAL_ZONE                    object
START_RENTAL_ZONE_HAL_ID              int64
END_RENTAL_ZONE                      object
END_RENTAL_ZONE_HAL_ID                int64
RENTAL_ZONE_HAL_SRC                  object
CITY_RENTAL_ZONE                     object
TECHNICAL_INCOME_CHANNEL             object
TRAVERSE                              int64
EXTRA_BOOKING_FEE                     int64
VEHICLE_MANUFACTURER_NAME            object
FUEL_TYPE_NAME                       object
CATEGORY                             object
POPULATION                      

In [20]:
#for booking date
data['BOOKING_DATE']= data.DATE_BOOKING.dt.date
data['BOOKING_YEAR']= data.DATE_BOOKING.dt.year
data['BOOKING_MONTH']= data.DATE_BOOKING.dt.month
data['BOOKING_WDAY']= data.DATE_BOOKING.dt.weekday

In [21]:
data['BOOKING_QUARTER']= data.DATE_BOOKING.dt.quarter

In [22]:
data['FROM_MONTH']= data.DATE_FROM.dt.month

In [23]:
data['UNTIL_MONTH']= data.DATE_UNTIL.dt.month

In [24]:
data['FROM_WDAY']= data.DATE_FROM.dt.weekday

In [25]:
data['UNTIL_WDAY']= data.DATE_UNTIL.dt.weekday

In [26]:
data['BOOKING_DATE'] = pd.to_datetime(data.BOOKING_DATE)

In [27]:
#data.apply(lambda x: len(x.unique()))

In [28]:
#data['VEHICLE_MANUFACTURER_NAME'].value_counts()

In [29]:
#data['VEHICLE_HAL_ID'].value_counts()

In [30]:
#data.describe()

In [31]:
#data.groupby('CUSTOMER_HAL_ID')['BOOKING_HAL_ID'].apply(np.unique)

In [32]:
#data.pivot_table(columns='CUSTOMER_HAL_ID')

In [33]:
#data['START_RENTAL_ZONE'].value_counts()

In [34]:
#db_data['RENTAL_ZONE_HAL_SRC'].value_counts()

In [35]:
#db_data.applymap(lambda x: not x or pd.isnull(x)).sum()

In [36]:
#db_data[['RENTAL_ZONE_HAL_SRC', 'DISTANCE']].plot()

In [37]:
#data['TECHNICAL_INCOME_CHANNEL'].value_counts(dropna=False)

In [38]:
#db_data.iloc[:,6:].plot()
#plt.show()
#db_data.isnull().sum()

In [39]:
#db_data[db_data.DISTANCE.isnull()].sort_index()
#0 and NaN vales?

In [40]:
#db_data.loc[db_data['DISTANCE'].isnull()].sum()

#data[data['DISTANCE']==0]

In [41]:
#db_data[db_data.TECHNICAL_INCOME_CHANNEL.isnull()].sort_index()
#what shold we do with this null values

In [42]:
data.DATE_BOOKING.max()

Timestamp('2017-05-16 05:01:56')

In [43]:
data.BOOKING_YEAR.value_counts().sort_index()

2013       897
2014    169872
2015    152722
2016    127674
2017     41448
Name: BOOKING_YEAR, dtype: int64

In [44]:
data.DATE_BOOKING.min()

Timestamp('2013-06-05 08:49:33')

In [45]:
data.DATE_FROM.min()

Timestamp('2014-01-01 00:30:00')

In [46]:
#one_tenth = db_data.sample(frac = .1, random_state=np.random.randint(10))

In [47]:
#one_tenth = one_tenth.sort_values(by=['DATE_BOOKING'], ascending=[True])

In [48]:
#one_tenth.head()

In [49]:
data.head()

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,...,FEDERAL_STATE,BOOKING_DATE,BOOKING_YEAR,BOOKING_MONTH,BOOKING_WDAY,BOOKING_QUARTER,FROM_MONTH,UNTIL_MONTH,FROM_WDAY,UNTIL_WDAY
0,17842196,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-06-05 08:49:33,2014-01-12 13:00:00,2014-01-12 14:30:00,Nein,Nein,14,...,Nordrhein-Westfalen,2013-06-05,2013,6,2,2,1,1,6,6
1,19923967,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-09-15 12:19:14,2014-01-05 14:00:00,2014-01-05 15:30:00,Nein,Nein,0,...,Nordrhein-Westfalen,2013-09-15,2013,9,6,3,1,1,6,6
2,20453330,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-10-19 13:25:38,2014-03-16 18:15:00,2014-03-16 19:30:00,Nein,Nein,17,...,Nordrhein-Westfalen,2013-10-19,2013,10,5,4,3,3,6,6
3,20799835,100012,150359,D564D3A1EA8BF096581556520B8F6582BEB65EFD,2013-11-10 14:10:26,2014-01-06 17:00:00,2014-01-06 23:45:00,Nein,Nein,16,...,Nordrhein-Westfalen,2013-11-10,2013,11,6,4,1,1,0,0
4,20799864,100012,150359,D564D3A1EA8BF096581556520B8F6582BEB65EFD,2013-11-10 14:13:25,2014-01-13 17:00:00,2014-01-13 23:45:00,Nein,Nein,16,...,Nordrhein-Westfalen,2013-11-10,2013,11,6,4,1,1,0,0


In [50]:
#data.columns

In [51]:
year = data[(data.BOOKING_YEAR < 2017) & (data.BOOKING_YEAR > 2013)]
year.head()

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,...,FEDERAL_STATE,BOOKING_DATE,BOOKING_YEAR,BOOKING_MONTH,BOOKING_WDAY,BOOKING_QUARTER,FROM_MONTH,UNTIL_MONTH,FROM_WDAY,UNTIL_WDAY
8,21378191,100012,150359,0FC7F06DDDD951AF7009288F8D4B9060925ED107,2014-01-02 17:54:09,2014-01-03 09:00:00,2014-01-03 22:30:00,Nein,Nein,1001,...,Nordrhein-Westfalen,2014-01-02,2014,1,3,1,1,1,4,4
9,21394268,100012,150359,0646C8373257083080F514A1E7B483BE1020D949,2014-01-04 15:07:28,2014-01-04 16:30:00,2014-01-04 17:30:00,Nein,Nein,2,...,Nordrhein-Westfalen,2014-01-04,2014,1,5,1,1,1,5,5
10,21395020,100012,150359,BEF6E7452694AB944BA5E2FADFA107C29D6DC111,2014-01-04 16:23:01,2014-01-04 17:45:00,2014-01-04 18:45:00,Nein,Nein,4,...,Nordrhein-Westfalen,2014-01-04,2014,1,5,1,1,1,5,5
11,21404210,100012,150359,B38DC9BA3EF8BB7717EA1B1EF91FFECC05D60567,2014-01-05 23:02:27,2014-01-05 23:15:00,2014-01-06 00:30:00,Nein,Nein,33,...,Nordrhein-Westfalen,2014-01-05,2014,1,6,1,1,1,6,0
12,21406248,100012,150359,31A345F78FE6B781182BEFFCA97779712E585472,2014-01-06 09:45:36,2014-01-07 10:00:00,2014-01-07 14:00:00,Nein,Nein,36,...,Nordrhein-Westfalen,2014-01-06,2014,1,0,1,1,1,1,1


In [52]:
#phdays = pd.read_csv("publicHolidays.csv", sep=',', header=0)
#phdays.head(9)

In [53]:
#phdays['BOOKING_DATE'] = pd.to_datetime(data.BOOKING_DATE)
#phdays['BOOKING_DATE']= phdays.BOOKING_DATE.dt.date"""

In [54]:
#data = pd.merge(data, phdays, on ='BOOKING_DATE')
#data.head()

In [55]:
#data.PUBLIC_HOLIDAYS.value_counts()

In [56]:
#col = ['PUBLIC_HOLIDAYS']
#data.drop(col, inplace=True, axis=1)
#data.head()"""